In [1]:
import pandas as pd

In [2]:
liens = pd.read_excel("liens.xlsx")
redemptions = pd.read_excel("redemptions.xlsx")
liens.head()

,Unique ID,County,Year,LLC,Block/Lot,Block,Lot,Qualifier,Adv #,MUA Acct # / Parcel ID,...,Total.22,Sub Type.23,Sub Date.23,Total.23,Sub Type.24,Sub Date.24,Total.24,Sub Type.25,Sub Date.25,Total.25
0,9169,Asbury Park,2014,TTLBL,505-19-,505,19,NaN,88.0,04 00505-0000-00019,...,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0
1,9170,Asbury Park,2014,TTLBL,604-11-,604,11,NaN,120.0,04 00604-0000-00011,...,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0
2,9171,Asbury Park,2014,TTLBL,901-16-,901,16,NaN,162.0,04 00901-0000-00016,...,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0
3,9172,Asbury Park,2014,TTLBL,902-2-,902,2,NaN,163.0,04 00902-0000-00002,...,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0
4,9173,Asbury Park,2014,TTLBL,902-33-,902,33,NaN,171.0,04 00902-0000-00033,...,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,0.0


In [3]:
w = redemptions.loc[redemptions["Cash In Date (Deposit Date)"].notnull(), ["Unique ID", "Cash In Date (Deposit Date)", "Total Received"]]
to_merge = w.groupby("Unique ID").min()

In [4]:
for_db = liens.merge(to_merge, on="Unique ID")

In [5]:
names = {
    "Unique ID": "lien_id",
    "County": "county",
    "Year": "year",
    "LLC": "llc",
    "Block": "block",
    "Lot": "lot",
    "Qualifier": "qualifier",
    "Adv #": "advertisement_number",
    "MUA Acct # / Parcel ID": "mua_number",
    "Certificate #": "certificate_number",
    "Lien Type": "lien_type",
    "List Item": "list_item",
    "Current Owner": "current_owner",
    "Longitude": "longitude",
    "Latitude": "latitude",
    "Assessed Value": "assessed_value",
    "Tax Amount": "tax_amount",
    "Status": "status",
    "Address": "address",
    "Cert FV": "certificate_face_value",
    "Winning Bid": "winning_bid_percentage",
    "Premium": "premium",
    "Sale Date": "sale_date",
    "Recording Fee": "recording_fee",
    "Recording Date": "recording_date",
    "Search Fee": "search_fee",
    "Year end penalty": "year_end_penalty",
    "Flat rate": "flate_rate",
    "Cert int": "cert_int",
    "Total subs paid": "total_subs_paid",
    "Total cash out": "total_cash_out",
    "Total cash recieved": "total_cash_received",
    "Total principal paid": "total_principal_paid",
    "Total actual interest": "total_actual_interest",
    "Total legal fees": "total_legal_fees",
    "Total Principal Balance": "total_principal_balance",
    "Notes": "notes",
    "Cash In Date (Deposit Date)": "redemption_date",
    "Total Received": "redemption_amount",
}

for_db.rename(names, axis=1, inplace=True)

In [6]:
liens_to_import = for_db[names.values()]

In [7]:
subs_to_melt = for_db.drop(list(names.values())[1:] + ["Block/Lot"], axis=1)
subs = pd.DataFrame()
for i in range(26):
    sub_type = "Sub Type"
    sub_date = "Sub Date"
    total = "Total"
    if i != 0:
        sub_type += f".{i}"
        sub_date += f".{i}"
        total += f".{i}"
    temp = subs_to_melt[["lien_id", sub_type, sub_date, total]]
    temp.columns = ["lien_id", "Sub Type", "Sub Date", "Total"]
    subs = subs.append(temp, sort=False)

In [8]:
subs_to_import = subs.sort_values("lien_id").reset_index(drop=True)
subs_to_import = subs_to_import.loc[subs_to_import["Total"] > 0, :].copy()

In [9]:
writer = pd.ExcelWriter("liens_to_import.xlsx", engine="xlsxwriter")
liens_to_import.to_excel(writer, sheet_name="liens", index=False)
subs_to_import.to_excel(writer, sheet_name="subs", index=False)

In [10]:
writer.save()